In [60]:
import numpy as np
import pandas as pd

In [61]:
ssi = pd.read_csv('./data/ssi_ca.csv')
ssi.head(3)

,Year,State,HAI,Operative_Procedure,Facility_ID,Facility_Name,Hospital_Category_RiskAdjustment,Hospital_Type,County,Procedure_Count,Infection_Count,Predicted_Infection_Count,SIR,SIR_CI_95_Lower_Limit,SIR_CI_95_Upper_Limit,Comparison,SIR_2015,"On Track Toward 2020 Goal (0.88, 2017)",Notes
0,2017.0,California,"Healthcare Associated Infection, Surgical Site...",STATE OF CALIFORNIA POOLED DATA-Abdominal aort...,NaN,NaN,NaN,NaN,NaN,515.0,2.0,3.502,0.57,0.10,1.89,Same,1.07,Yes,NaN
1,2017.0,California,"Healthcare Associated Infection, Surgical Site...",STATE OF CALIFORNIA POOLED DATA-Appendix surgery,NaN,NaN,NaN,NaN,NaN,29595.0,116.0,113.323,1.02,0.85,1.22,Same,0.98,No,NaN
2,2017.0,California,"Healthcare Associated Infection, Surgical Site...","STATE OF CALIFORNIA POOLED DATA-Bile duct, liv...",NaN,NaN,NaN,NaN,NaN,7975.0,255.0,211.450,1.21,1.07,1.36,Worse,0.97,No,NaN


In [62]:
ssi.shape

(6841, 19)

In [63]:
ssi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6841 entries, 0 to 6840
Data columns (total 19 columns):
 #   Column                                  Non-Null Count  Dtype  
---  ------                                  --------------  -----  
 0   Year                                    6837 non-null   float64
 1   State                                   6837 non-null   object 
 2   HAI                                     6837 non-null   object 
 3   Operative_Procedure                     6492 non-null   object 
 4   Facility_ID                             6809 non-null   float64
 5   Facility_Name                           6809 non-null   object 
 6   Hospital_Category_RiskAdjustment        6809 non-null   object 
 7   Hospital_Type                           6810 non-null   object 
 8   County                                  6809 non-null   object 
 9   Procedure_Count                         6837 non-null   float64
 10  Infection_Count                         6837 non-null   floa

In [64]:
ssi.isna().sum()/len(ssi) * 100

Year                                       0.058471
State                                      0.058471
HAI                                        0.058471
Operative_Procedure                        5.101593
Facility_ID                                0.467768
Facility_Name                              0.467768
Hospital_Category_RiskAdjustment           0.467768
Hospital_Type                              0.453150
County                                     0.467768
Procedure_Count                            0.058471
Infection_Count                            0.058471
Predicted_Infection_Count                  0.058471
SIR                                       47.580763
SIR_CI_95_Lower_Limit                     47.580763
SIR_CI_95_Upper_Limit                     47.580763
Comparison                                47.580763
SIR_2015                                  52.083029
On Track Toward 2020 Goal (0.88, 2017)    47.580763
Notes                                     66.963894
dtype: float

In [65]:
columns_of_interest = [
    'Operative_Procedure', 'Facility_ID',
    'Hospital_Type', 'Procedure_Count',
    'Infection_Count'
]

ssi_raw = ssi.copy()
ssi = ssi[columns_of_interest]
ssi.columns

Index(['Operative_Procedure', 'Facility_ID', 'Hospital_Type',
       'Procedure_Count', 'Infection_Count'],
      dtype='object')

In [66]:
ssi.isna().sum()/len(ssi) * 100

Operative_Procedure    5.101593
Facility_ID            0.467768
Hospital_Type          0.453150
Procedure_Count        0.058471
Infection_Count        0.058471
dtype: float64

In [67]:
ssi.dropna(axis=0, inplace=True)
ssi.head()

,Operative_Procedure,Facility_ID,Hospital_Type,Procedure_Count,Infection_Count
373,Abdominal aortic aneurysm repair,60000002.0,Community (125-250 Beds),10.0,0.0
374,Abdominal aortic aneurysm repair,110000058.0,Community (125-250 Beds),1.0,0.0
375,Abdominal aortic aneurysm repair,60000074.0,Community (125-250 Beds),4.0,0.0
376,Abdominal aortic aneurysm repair,930000008.0,Community (>250 Beds),4.0,1.0
377,Abdominal aortic aneurysm repair,120001330.0,Community (<125 Beds),1.0,0.0


In [68]:
# Querying only Community Type Hospitals:
ssi = ssi.query('Hospital_Type != "Pediatric" and Hospital_Type != "Major Teaching" and Hospital_Type != "Critical Access" ')

ssi['Hospital_Type'] = ssi['Hospital_Type'].str.replace('Community', '')
ssi['Hospital_Type'] = ssi['Hospital_Type'].str.replace('(', '')
ssi['Hospital_Type'] = ssi['Hospital_Type'].str.replace(')', '')
ssi['Hospital_Type'] = ssi['Hospital_Type'].str.replace('>250', '250+')
ssi['Hospital_Type'] = ssi['Hospital_Type'].str.replace('<125', 'Less than 125')

# Fixing 'Facility ID' data type:
ssi['Facility_ID'] = ssi['Facility_ID'].astype('int64').astype('object')

# Calculating SSI (in %):
ssi['SSI_ratio'] = ssi['Infection_Count'] / ssi['Procedure_Count'] * 100

# Create a dictionary to map procedures to categories
procedure_categories = {
    'Gastrointestinal Procedures': [
      'Appendix surgery', 'Bile duct, liver or pancreatic surgery',
      'Colon surgery', 'Gallbladder surgery', 'Gastric surgery',
      'Rectal surgery', 'Small bowel surgery', 'Spleen surgery'
     ],
    'Cardiac Procedures': [
        'Cardiac surgery', 'Coronary bypass, chest and donor incisions',
        'Coronary bypass, chest incision only', 'Heart transplant', 'Pacemaker surgery'
    ],
    'Obstetric and Gynecological Procedures': [
        'Cesarean section', 'Hysterectomy, abdominal',
        'Hysterectomy, vaginal', 'Ovarian surgery'
    ],
    'Orthopedic Procedures': [
        'Hip prosthesis', 'Knee prosthesis', 'Open reduction of fracture',
        'Spinal fusion'
    ],
    'Transplant Procedures': ['Kidney surgery', 'Kidney transplant', 'Liver transplant'],
    'Other Procedures': [
        'Exploratory abdominal surgery (laparotomy)',
        'Laminectomy', 'Thoracic surgery',
        'Abdominal aortic aneurysm repair'
    ]
}

# Function to assign categories based on procedure
def categorize_procedure(procedure):
    for category, procedures in procedure_categories.items():
        if procedure in procedures:
            return category
    return 'Other'

# Add a new 'Category' column based on procedure
ssi['Procedure_Category'] = ssi['Operative_Procedure'].apply(categorize_procedure)

In [69]:
order_for_columns = [
    'Facility_ID', 'Hospital_Type',
    'Procedure_Category', 'Operative_Procedure',
    'Procedure_Count', 'Infection_Count', 
    'SSI_ratio'
]

ssi = ssi[order_for_columns]
ssi = ssi.sort_values(by='Procedure_Category', ascending=True)

In [70]:
# Verification:
ssi.head()

,Facility_ID,Hospital_Type,Procedure_Category,Operative_Procedure,Procedure_Count,Infection_Count,SSI_ratio
5542,110000067,125-250 Beds,Cardiac Procedures,Pacemaker surgery,93.0,0.0,0.0
5460,120000404,Less than 125 Beds,Cardiac Procedures,Pacemaker surgery,1.0,0.0,0.0
5461,30000901,Less than 125 Beds,Cardiac Procedures,Pacemaker surgery,26.0,0.0,0.0
5462,40000113,125-250 Beds,Cardiac Procedures,Pacemaker surgery,29.0,0.0,0.0
5464,230000010,250+ Beds,Cardiac Procedures,Pacemaker surgery,128.0,0.0,0.0


In [71]:
# Seting up a benchmark for SSI:
ssi['Performance'] = ssi['SSI_ratio'].apply(
    lambda x:
      'Bad' if (x >= 3)
      else 'Good'
)

In [72]:
# Verification:
ssi.head()

,Facility_ID,Hospital_Type,Procedure_Category,Operative_Procedure,Procedure_Count,Infection_Count,SSI_ratio,Performance
5542,110000067,125-250 Beds,Cardiac Procedures,Pacemaker surgery,93.0,0.0,0.0,Good
5460,120000404,Less than 125 Beds,Cardiac Procedures,Pacemaker surgery,1.0,0.0,0.0,Good
5461,30000901,Less than 125 Beds,Cardiac Procedures,Pacemaker surgery,26.0,0.0,0.0,Good
5462,40000113,125-250 Beds,Cardiac Procedures,Pacemaker surgery,29.0,0.0,0.0,Good
5464,230000010,250+ Beds,Cardiac Procedures,Pacemaker surgery,128.0,0.0,0.0,Good


In [73]:
ssi.Facility_ID.nunique()

262

In [74]:
# Exporting:
ssi.to_csv('ssi_data.csv', index=False)